In [70]:
# Importing packages

import numpy as np
import pandas as pd
from time_converter import Time
import wget
import time
import json
import gzip
import re
import pandasql as psql

In [ ]:
filelist = {"usersdf":"users.json.gz","receiptsdf":"receipts.json.gz","brandsdf":"brands.json.gz"}

## Users data  
### Cleaning and Conversion

In [2]:
# Unzipping and converting json to csv
dfdata = []
with gzip.open("users.json.gz") as data:
    for i in data:
        dfdata.append(json.loads(i.strip()))
    user = pd.json_normalize(dfdata)

In [3]:
user.head()

,active,role,signUpSource,state,_id.$oid,createdDate.$date,lastLogin.$date
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,1609687530554,1.609688e+12
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12


In [ ]:
#user.info()

In [ ]:
#users.to_csv('users.csv')

####  This below function aims at extracting, processing, and cleaning date-related information from a specified column in a DataFrame

In [4]:
#This function is designed to process and clean a list of date-related data

def cleanDateLog(id_data, column=None):
    id_list = list(id_data[f'{column}'].to_dict().values())
    timeslist = []
    dateslist = []
    for i in range(len(id_list)):
        try: 
            
            date_parsed = id_list[i].replace('}', "").split(':')[1].replace(" ", "")
            date_converted = Time(int(date_parsed[:10]), 'posix').to('dt')
            format_date = "%m/%d/%Y"
            format_time = "%H:%M:%S"
            new_date = date_converted.strftime(format_date)
            new_time = date_converted.strftime(format_time)
            timeslist.append(new_time)
            dateslist.append(new_date)
        except AttributeError:
            
            date_parsed = str(id_list[i])
            dateslist.append(date_parsed)
            
    return dateslist

#### The purpose of below function is to extract ID strings from a specified column, and then parse out and clean the names enclosed within single quotes from each ID string. The extracted names are stored in a list and returned as the result of the function.

In [5]:
# This function cleans and process a list of strings containing IDs

def clean_id_cpg(id_data, column=None):
    id_list = list(id_data[column].to_dict().values())
    cleaned_list = []
    
    for id_item in id_list:
        parsed_names = re.findall("'([^']*)'", id_item)
        if parsed_names:
            cleaned_list.append(parsed_names[-1])
    
    return cleaned_list


#### This function performs  cleaning and conversion steps on the data using clean_id_cpg and cleanDateLog and then saves the cleaned data to a new CSV file.

In [6]:
def cleanUserData():    
    user = pd.read_csv('users.csv', index_col = 0)
    users = user.copy()

    users['_id'] = clean_id_cpg(id_data=users, column="_id")
    users['createdDate'] = cleanDateLog(id_data=users, column="createdDate")
    users["lastLogin"] = cleanDateLog(id_data=users, column="lastLogin")

    # coversion to datetime 
    users['createdDate'] = pd.to_datetime(users['createdDate'])
    users["lastLogin"] = pd.to_datetime(users['lastLogin'])
    users = users.rename(columns = {'_id':'user_id'}, inplace = False)
    users.to_csv('users_cleaned.csv')
    
    return users

In [7]:
users = cleanUserData()

In [8]:
users.head()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03,2021-01-03,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI


## Receipt  
### Cleaning and Conversion

In [9]:
# Concerting dates
def convert_dates(data_frame):
    new_df = data_frame.copy()
    column_list = list(new_df.columns)
    date_columns = column_list[3:10]
    date_columns.remove(column_list[8])
    
    for date_col in date_columns:
        new_df[date_col] = cleanDateLog(id_data=new_df, column=date_col)
        
    return new_df


In [10]:
# cleaning the ids column
def cleanReceipt():
    
    receipts = pd.read_csv('receipts.csv', index_col = 0)
    receipts = convert_dates(receipts)
    receipts['_id'] = cleanIdCpg(receipts, column = '_id')
    receipts = receipts.drop('rewardsReceiptItemList', axis = 1)
    
    return receipts

####  The goal of the below function is to clean and process date columns, convert IDs, and save the modified data back to the CSV file

In [11]:
def process_receipts():
    receipts = pd.read_csv('receipts.csv', index_col=0)
    receipts = convert_dates(receipts)
    receipts['_id'] = clean_id_cpg(receipts, column='_id')
    receipts = receipts.rename(columns={'_id': 'receipt_id'}, inplace=False)
    
    date_columns = ['createDate', 'dateScanned', 'finishedDate', 'purchaseDate', 'purchaseDate']
    for col in date_columns:
        receipts[col] = pd.to_datetime(receipts[col])
    
    print(receipts)
    receipts.to_csv('receipts.csv')
    
    return receipts


In [12]:
receipts = process_receipts()

                    receipt_id  bonusPointsEarned  \
0     5ff1e1eb0a720f0523000575              500.0   
1     5ff1e1bb0a720f052300056b              150.0   
2     5ff1e1f10a720f052300057a                5.0   
3     5ff1e1ee0a7214ada100056f                5.0   
4     5ff1e1d20a7214ada1000561                5.0   
...                        ...                ...   
1114  603cc0630a720fde100003e6               25.0   
1115  603d0b710a720fde1000042a                NaN   
1116  603cf5290a720fde10000413                NaN   
1117  603ce7100a7217c72c000405               25.0   
1118  603c4fea0a7217c72c000389                NaN   

                                bonusPointsEarnedReason createDate  \
0     Receipt number 2 completed, bonus point schedu... 2021-01-03   
1     Receipt number 5 completed, bonus point schedu... 2021-01-03   
2                            All-receipts receipt bonus 2021-01-03   
3                            All-receipts receipt bonus 2021-01-03   
4            

In [13]:
receipts.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03,2021-01-03,2021-01-03,01/03/2021,01/03/2021,500.0,2021-01-03,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03,2021-01-03,2021-01-03,01/03/2021,01/03/2021,150.0,2021-01-02,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,NaT,01/03/2021,nan,5.0,2021-01-03,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,2021-01-03,01/03/2021,01/03/2021,5.0,2021-01-03,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,2021-01-03,01/03/2021,01/03/2021,5.0,2021-01-02,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [15]:
df_data = []
with gzip.open("receipts.json.gz") as data:
    for i in data:
        df_data.append(json.loads(i.strip()))
    receiptsdf = pd.json_normalize(df_data)
receiptsdf.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12


In [16]:
receiptsdf=receiptsdf.rename(columns= {
        '_id.$oid' : 'receiptId',
        'createDate.$date' : 'receipt_createDate',
        'dateScanned.$date' : 'receipt_ScannedDate',
        'finishedDate.$date' : 'receipt_finishedDate',
        'modifyDate.$date' : 'receipt_modifyDate',
        'pointsAwardedDate.$date' : 'pointsAwardedDate',
        'purchaseDate.$date' : 'purchaseDate'})

In [17]:
# Checking for Nulls
print(f'Total Records: {len(receiptsdf["rewardsReceiptItemList"])}')
print(f'Null Records: {receiptsdf["rewardsReceiptItemList"].isnull().sum()}')

Total Records: 1119
Null Records: 440


In [18]:
# Removing all the Null values
receiptsdf['rewardsReceiptItemList']= receiptsdf['rewardsReceiptItemList'].replace(np.nan,0)

In [19]:
# After Removal of Null values
print(f'Total Records: {receiptsdf["rewardsReceiptItemList"].count()}')
print(f'Null Records: {receiptsdf["rewardsReceiptItemList"].isnull().sum()}')

Total Records: 1119
Null Records: 0


### Flattening of nested JSON which is inside the Receipt  into a new dataframe.  
#### Created new dataframe ordersdf which has products and quantity along with the prices for each item

In [30]:
cntr=1
receipt_ids=[]
data=[]
for i in range(len(receiptsdf.rewardsReceiptItemList)):
    if receiptsdf.rewardsReceiptItemList[i] != 0:
        for items in receiptsdf.rewardsReceiptItemList[i]:
            items['receipt_id']=str(cntr)
            data.append(items)
        receipt_ids.append(items['receipt_id'])
        cntr = cntr + 1
    else:
        receipt_ids.append('0')

ordersdf=pd.DataFrame.from_dict(data)
ordersdf.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
receipts.drop(columns=['rewardsReceiptItemList'],inplace=True)

## Brands  
### Cleaning and Conversion

In [27]:
df_data = []
with gzip.open("brands.json.gz") as data:
    for i in data:
        df_data.append(json.loads(i.strip()))
    brands = pd.json_normalize(df_data)

In [29]:
brands=brands.rename(columns={'_id.$oid':'brand_id',brands.columns[-3]: 'cpg_id','cpg.$ref' : 'cpg_ref'})
brands.head(5)

,barcode,category,categoryCode,name,topBrand,brand_id,cpg_id,cpg_ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


# Part 3: Data Quality Checks

In [35]:
users.head()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03,2021-01-03,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI


In [39]:
users.isna().sum()

user_id          0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

*As shown above 3 columns have Null values which might affect the analysis. So, it is always a good practice to remove the Null values or replace the Null values with Mean or Median to avoid misinterpretations.*

### Checking for Duplicates

In [40]:
users.nunique()

user_id         212
active            2
createdDate      42
lastLogin        30
role              2
signUpSource      2
state             8
dtype: int64

In [43]:
users.shape

(495, 7)

*Out of 495 only 212 user_id are unique. This creates a major data quality issue since for users table user_id is primary key and for it to be primary key all the values should be unique. Hence this will create issue will joining the tables and also while analysis.*

In [47]:
receipts.isna().sum()

receipt_id                   0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate            0
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

*7 out of 14 columns have missing value. This needs to removed or addressed by replacing Null values with Mean or Median.*

In [49]:
print(len(receipts))
receipts.nunique()

1119


receipt_id                 1119
bonusPointsEarned            12
bonusPointsEarnedReason       9
createDate                   49
dateScanned                  49
finishedDate                 36
modifyDate                   42
pointsAwardedDate            39
pointsEarned                119
purchaseDate                 69
purchasedItemCount           50
rewardsReceiptStatus          5
totalSpent                   94
userId                      258
dtype: int64

*receipt_id serves as a primary key and it does not have duplicate values and Null values.*

In [50]:
print(len(brands))

1167


In [51]:
brands.isna().sum()

barcode           0
category        155
categoryCode    650
name              0
topBrand        612
brand_id          0
cpg_id            0
cpg_ref           0
brandCode       234
dtype: int64

In [52]:
brands.nunique()

barcode         1160
category          23
categoryCode      14
name            1156
topBrand           2
brand_id        1167
cpg_id           196
cpg_ref            2
brandCode        897
dtype: int64

*brand_id(primary key) has no Null and duplicate value which is ideally perfect.  
However almost half values of topBrand are missing.*

*Rewards receipt items list uses a nested dictionary, which can make data cleansing more difficult, particularly when a receipt item lacks an associated id that would allow us to identify which user's receipt item it was.*  

 *Purchased Date has NAT values which caused issue if time series analysis has to done.*


### In Orders data 173 products dont have description. This can be misleading and can hamper analysis.

In [65]:
itemnotfound=Ordersdf[Ordersdf['description'] == "ITEM NOT FOUND"]

In [66]:
itemnotfound.groupby("description")['description'].count()

description
ITEM NOT FOUND    173
Name: description, dtype: int64

### 54 brand code has the value same as barcode value.

In [75]:
samecode = brands[brands['barcode'] == brands['brandCode']]

In [77]:
samecode.head(5)

,barcode,category,categoryCode,name,topBrand,brand_id,cpg_id,cpg_ref,brandCode
13,511111205012,Magazines,NaN,Entertainment Weekly,NaN,5d6413156d5f3b23d1bc790a,5d5d4fd16d5f3b23d1bc7905,Cogs,511111205012
27,511111105329,Magazines,NaN,Elegant Homes Magazine,NaN,5d66d71fa3a018093ab34728,5d5d4fd16d5f3b23d1bc7905,Cogs,511111105329
44,511111505365,Magazines,NaN,Magnolia Journal Magazine,NaN,5d66d94d6d5f3b6188d4f04b,5d5d4fd16d5f3b23d1bc7905,Cogs,511111505365
64,511111805854,Health & Wellness,NaN,ONE A DAY® WOMENS,False,5da609991dda2c3e1416ae90,53e10d6368abd3c7065097cc,Cogs,511111805854
134,511111305569,Health & Wellness,NaN,AFRIN® NO DRIP PUMP MISTS,NaN,5da60576a60b87376833e349,53e10d6368abd3c7065097cc,Cogs,511111305569


In [76]:
print(len(samecode))

54


### Purchase date is later than receipt create Date in Receipts Dataset

In [81]:

print(psql.sqldf("""select purchaseDate,dateScanned
                    from receipts where purchaseDate > dateScanned
                    """))

                  purchaseDate                 dateScanned
0   2021-02-03 00:00:00.000000  2021-01-03 00:00:00.000000
1   2021-02-03 00:00:00.000000  2021-01-03 00:00:00.000000
2   2021-02-05 00:00:00.000000  2021-01-05 00:00:00.000000
3   2021-02-07 00:00:00.000000  2021-01-07 00:00:00.000000
4   2021-02-08 00:00:00.000000  2021-01-08 00:00:00.000000
5   2021-02-11 00:00:00.000000  2021-01-11 00:00:00.000000
6   2021-02-13 00:00:00.000000  2021-01-13 00:00:00.000000
7   2021-02-13 00:00:00.000000  2021-01-13 00:00:00.000000
8   2021-02-14 00:00:00.000000  2021-01-14 00:00:00.000000
9   2021-02-20 00:00:00.000000  2021-01-20 00:00:00.000000
10  2021-02-28 00:00:00.000000  2021-01-29 00:00:00.000000
11  2021-03-01 00:00:00.000000  2021-02-01 00:00:00.000000
12  2021-03-08 00:00:00.000000  2021-02-08 00:00:00.000000
